# Import Packages

In [57]:
import os

import pandas as pd
from tqdm.auto import tqdm

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_distances
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from gensim.models import Word2Vec

sw_indo = stopwords.words("indonesian") + list(punctuation)

# Read Dataset

In [58]:
df = pd.read_csv("./data/kompas.csv")
df.head()

,teks
0,Ginandjar Tetap Ditahan. Jaksa Agung Dilaporka...
1,Jakarta Dikangkangi Para Preman\nKALAU tak pun...
2,Penyimpangan di Setpres Seolah Terjadi Sekaran...
3,"Dibayarkan, Rapel Kenaikan Gaji Pegawai Pos\nK..."
4,"Stop Kekerasan, Elite agar Duduk Bersama\nSeju..."


# Info Dataset

In [17]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2008 entries, 0 to 2007
Data columns (total 1 columns):
teks    2008 non-null object
dtypes: object(1)
memory usage: 9.5 MB


# Preprocessing

In [18]:
def text_preprocessing(text: str) -> str:

    text = text.lower()
    text = " ".join([word for word in word_tokenize(text) if word not in sw_indo])
    return text

# Vectorizer

In [19]:
tfidf = TfidfVectorizer(
    ngram_range=(1, 2), 
    tokenizer=word_tokenize, 
    preprocessor=text_preprocessing
)
tfidf_matrix = tfidf.fit_transform(df.teks.values)

# Document Similarity

In [52]:
def document_similarity(documents, compared_doc,model, tfidf_matrix, topN=10):
    idx_documents = cosine_distances(model.transform([compared_doc]).toarray(), tfidf_matrix)[0].argsort()[:10]
    
    return [documents[idx][:100] for idx in idx_documents]

document_similarity(df.teks, "christiano ronaldo", tfidf, tfidf_matrix)

['Arsenal dan Liverpool Siap Melejit\n\nArsenal dan Liverpool bersiap-siap melejit meninggalkan juara be',
 'Italia, Perancis, Jerman, Brasil Sukses dalam Uji Coba\n\nEmpat tim mapan, Italia, Perancis, Jerman, d',
 'Kecelakaan di Tol Cacing, Tiga Tewas\n\nTiga orang tewas di tempat dan 16 orang lainnya luka-luka akib',
 ' Timtim: Kalau UNTAET Pergi\n\nKALAU UNTAET pergi, kami bagaimana?"......... ....Kalimat itu diucapkan',
 'Joseph Estrada Muncul Kembali\n\nMeski telah disingkirkan dari kekuasaannya, mantan Presiden Filipina ',
 'Deportivo La Coruna Lapangkan Jalan Bayer Leverkusen\n\nDi luar dugaan, Deportivo La Coruna dipaksa me',
 'Empat Perwira Penerbang Tewas\n\nDua pesawat tempur jenis Hawk MK-53 "bersenggolan" saat latihan di ar',
 'Forum Dialog Barat-Islam   Tidak Ada Terorisme Berbasis Agama\n\nTidak ada satu agama pun di dunia yan',
 'Pemerintah Sediakan Rp 33 Milyar untuk Bangun Rumah di Poso\n\nPemerintah pusat mengalokasikan dana se',
 'Kontak Tembak  TNI-GAM, Lima Orang 

# Sanity Check

In [55]:
example_documents = """
Beberapa gejala infeksi virus corona (Covid-19) adalah sakit tenggorokan, demam, dan batuk. Sekilas memang mirip dengan gejala flu. Namun para ahli menegaskan, virus corona berbeda dengan flu biasa dari sisi risiko dan penularan.

Virus corona terbukti menimbulkan kematian sebesar 3,5 persen dari kasus pasien positif.

Angka ini jauh lebih tinggi daripada potensi kematian pada flu biasa, yang hanya 0,1 persen dari pasien.

Itu berarti, rata-rata risiko kematian pada virus corona lebih tinggi 20 kali lipat dari flu biasa.

"Terdapat ketidakpastian besar terkait tingkat kematian virus corona. Kemungkinannya bervariasi tergantung pada kualitas layanan kesehatan lokal," kata Profesor Biologi Sistem Komputasi di University College London, Francois Balloux.

Para ahli mengatakan bahaya sebenarnya dari virus corona bukanlah korban jiwa. Namun rumah sakit dapat kewalahan oleh jumlah kasus yang memerlukan rawat inap dan ventilasi untuk mendukung pernapasan.

Sebuah analisis terhadap 45 ribu kasus di China, sebagian besar kematian adalah orang tua, yakni 14,8 persen dari korban meninggal usianya lebih dari 80-an).

Namun demikian, penelitian juga menyebutkan 41 persen kasus serius terjadi di bawah usia 50 tahun.

Sedangkan 27 persen kasus serius menimpa pasien dengan usia di atas 65 tahun.

"Memang benar bahwa jika Anda lebih tua Anda berisiko lebih besar, tetapi kasus-kasus serius juga dapat terjadi pada orang yang relatif muda tanpa kondisi (sakit) sebelumnya," kata Wakil Menteri Kesehatan Prancis Jerome Salomon.

Para ahli penyakit memperkirakan setiap penderita virus corona dapat menginfeksi dua hingga tiga orang lainnya.

Tingkat penularan itu jauh lebih tinggi dari flu biasa yang menginfeksi 1,3 orang baru untuk setiap pasien.
"""

document_similarity(df.teks, example_documents, tfidf, tfidf_matrix)

['Masyarakat Harus Waspadai Leptospirosis\n\nDinas Kesehatan DKI Jakarta bekerja sama dengan Departemen ',
 'Leptospirosis yang Mematikan Itu\n\nSUHANDI (42) mengira sakitnya cuma demam biasa. Tubuhnya menggigil',
 '20 Tahun AIDS   Masih Banyak yang Belum Terjawab\n\nDALAM 20 tahun terakhir sejak kasus-kasus pertama ',
 'Korban Banjir Terserang Berbagai Penyakit\n\nBerbagai penyakit, terutama infeksi saluran pernapasan at',
 'LAPORAN IPTEK Terapi Gen, untuk Kualitas Hidup Manusia\n\nSETELAH era mikrobiologi dan sintesa zat akt',
 'Kotak Pandora Rekayasa Genetika  LAPORAN IPTEK\nLUAR biasa. Lebih dari 40 ton benih kapas transgenik ',
 'RS Panti Wilasa Terbakar, Ratusan Pasien Diungsikan\nRatusan pasien Rumah Sakit (RS) Panti Wilasa di ',
 'Mengapa Belanda Pilih Euthanasia?\nBERITA dari Belanda kembali menghiasi halaman pertama media cetak ',
 'Wapres tentang Pendidikan dan Kesehatan:  Indonesia Sangat Tertinggal\n\nIndonesia sangat tertinggal d',
 'Kalau Belasan Orang Gila Kabur\n\nKALA

# Using word2vec

In [60]:
sentences = [word_tokenize(text_preprocessing(rev)) for rev in tqdm(df.teks)]

In [61]:
model = Word2Vec(sentences, size=128, window=5, min_count=3, workers=4, iter=1000)

KeyboardInterrupt: 